# Support Vector Machines (SMVs)

One of the most popular ML models that can be used for linear or non-linear classification, regression, and outlier detection. SMVs are particularly for classification on complex small- or medium-sized datasets.

### Table of Content
#### 1. Linear SVM Classification
#### 2. Non-Linear SVM Classification
#### 3. SVM Regression


# 1. Linear SVM Classification

The basic idea is to find a decision boundary that not only separates the classes but stays as far away from the closest training instances as possible. Thus, a SVM tries to build the widest street possible between the classes (**large margin classification**).

**Soft Margin Classification**
Since it is not always possible to find a decision boundary that perfectly separates the classes and has a wide margin (for example when there are outliers), we can build a more flexible model that allows for instances to end up "in the middle of the road" or on the other side (which is a **margin violation**) but still tries to keep the road as wide as possible.

The **hyperparameter C** is used to control how wide the street will be and thus influences the amount of margin error. Again, we have a trade-off: a model with only a few margin violations might generalize worse than one with a wider street and more violations.